In [3]:
from pykiwoom.kiwoom import *

kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

kospi = kiwoom.GetCodeListByMarket('0')
kosdaq = kiwoom.GetCodeListByMarket('10')
etf = kiwoom.GetCodeListByMarket('8')

print(len(kospi), kospi)
print(len(kosdaq), kosdaq)
print(len(etf), etf)

AttributeError: 'QAxWidget' object has no attribute 'OnReceiveTrData'

In [49]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [166]:
#특정 날짜 23년 2월 2일 크롤링
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23&wr_id=2190#section1'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml')    

In [167]:
#특정 날짜 23년 2월 2일 크롤링
df = pd.DataFrame(columns=['comment'])
for i in range(len(bs.select('.cmt_contents'))):
    comment = bs.select('.cmt_contents')[i].text
    df.loc[i] = [comment]

In [168]:
df

,comment
0,\n\n \n
1,\n\n \n
2,\n\n \n
3,\n\n \n
4,\n\n전일 미국증시부터 살펴보고 가겠습니다. \n
...,...
81,\n\n\n
82,\n\n\n
83,\n\n \n
84,"\n\n코스피 상승, 코스닥 상승 거래소 : 개인 매도 / 외국인 매수 / 기관 매..."


In [4]:
#게시판에서 각 게시물 주소얻기 
url = 'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list23'
response = requests.get(
    url,
    headers={
        'User-Agent': 'Mozilla 5.0'
    }
    )
bs = BeautifulSoup(response.text, 'lxml') 

In [198]:
len(bs.select('table>tr'))

33

In [200]:
#1페이지 링크 얻기
for i in range(len(bs.select('table>tr'))):
    date = bs.select('table>tr')[i].select('dd')[1].text
    url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [5]:
#해당 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
for page_num in range(1,4):
    url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list22&page={page_num}'
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
        )
    bs = BeautifulSoup(response.text, 'lxml') 

    for i in range(len(bs.select('table>tr'))):
        date = bs.select('table>tr')[i].select('dd')[1].text
        url_link = bs.select('table>tr')[i].select('a')[0].get('href')

In [8]:
def get_info_from_url(url):
    response = requests.get(
        url,
        headers={
        'User-Agent': 'Mozilla 5.0'
        }
    )
    bs = BeautifulSoup(response.text, 'lxml') 
    return bs

In [9]:
#모든 년도 모든 페이지에 날짜 url 얻기
#마지막 페이지 찾는 작업 필요 
df = pd.DataFrame()
for year in tqdm(range(19,24)):
    for page_num in tqdm(range(1,4)):
        url = f'http://www.fantastock.co.kr/bbs/board.php?bo_table=free_list{year}&page={page_num}'
        bs = get_info_from_url(url)  
        info_from_bs = bs.select('table>tr')
        for bs in info_from_bs:
            try :
                date = bs.select('dd')[1].text
                url_link = bs.select('a')[0].get('href')
                bs = get_info_from_url(url_link)
                data_from_url = bs.select('.cmt_contents')

                for comments in data_from_url:
                    comment = comments.text

                    temp = pd.DataFrame({
                        'date': date,
                        'url': url_link,
                        'comment': comment
                        }, index=[0])

                    df = pd.concat([df, temp], axis=0)
            except IndexError:
                print(1)

100%|██████████| 5/5 [02:36<00:00, 31.31s/it]

1
1


In [41]:
#파일로 저장
df.to_csv('data.csv', index=False, encoding='utf-8-sig')

In [42]:
#파일 불러오기
df = pd.read_csv('./data.csv')

In [44]:
#매수 포함 가져오기
df = df[df['comment'].str.contains('매수', na =False)]

In [45]:
#현재시간 포함 가져오기
df = df[df['comment'].str.contains('현재시간', na =False)]

In [51]:
#공백 지우기
df['comment']=df.comment.apply(lambda x : re.sub('\\n',' ',x) )

In [ ]:
soojung['refined_comments'] = soojung.comments.apply(lambda x: re.sub('[ㄱ-ㅎㅏ-ㅣ^!.,]', '', x))

In [98]:
df['slicing_comment'] = df.comment.apply(lambda x : re.findall('\d+분 (\w+)',x))

In [109]:
df

,date,url,comment,slicing_comment
18,2019.12.27,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 00분 에스맥 현재가 1105원 기준 매수!!,[에스맥]
20,2019.12.27,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 01분 화천기계 현재가 3180원 기준 매수!!,[화천기계]
22,2019.12.27,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 02분 루미마이크로 현재가 1970원 기준 매수!!,[루미마이크로]
123,2019.12.26,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 00분 한국화장품 현재가 11400원 기준 매수!!,[한국화장품]
125,2019.12.26,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 01분 디앤씨미디어 현재가 16500원 기준 매수!!,[디앤씨미디어]
...,...,...,...,...
41578,2022.12.22,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 3분 팬오션 현재가 6050원 기준 매수!!,[팬오션]
41645,2022.12.21,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 6분 세종메디칼 현재가 3450원 기준 매수!!,[세종메디칼]
41647,2022.12.21,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 7분 청담글로벌 현재가 11350원 기준 매수!!,[청담글로벌]
41797,2022.12.19,http://www.fantastock.co.kr/bbs/board.php?bo_t...,▶ 현재시간 9시 0분 노터스 현재가 6250원 기준 매수!!,[노터스]
